In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf

from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '../lesson1/notMNIST_cor1.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (15000, 28, 28) (15000,)
Test set (16355, 28, 28) (16355,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (15000, 28, 28, 1) (15000, 10)
Test set (16355, 28, 28, 1) (16355, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [35]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(tf.add(conv, layer1_biases))
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden =  tf.nn.relu(tf.add(conv, layer2_biases))
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.add(tf.matmul(reshape, layer3_weights), layer3_biases))
    return tf.add(tf.matmul(hidden, layer4_weights), layer4_biases)
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [36]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.407704
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Minibatch loss at step 50: 1.484784
Minibatch accuracy: 56.2%
Validation accuracy: 44.2%
Minibatch loss at step 100: 0.451961
Minibatch accuracy: 93.8%
Validation accuracy: 70.6%
Minibatch loss at step 150: 1.211827
Minibatch accuracy: 75.0%
Validation accuracy: 75.6%
Minibatch loss at step 200: 0.653188
Minibatch accuracy: 75.0%
Validation accuracy: 76.1%
Minibatch loss at step 250: 0.199221
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%
Minibatch loss at step 300: 0.656584
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.660960
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Minibatch loss at step 400: 0.509909
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.623768
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.722943
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Mi

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    max_pol = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    hidden = tf.nn.relu(tf.add(max_pol, max_pol))
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    max_pol = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    hidden =  tf.nn.relu(tf.add(max_pol, layer2_biases))
    shape = hidden.get_shape().as_list()
#     print(shape[1], shape[2])
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.add(tf.matmul(reshape, layer3_weights), layer3_biases))
    return tf.add(tf.matmul(hidden, layer4_weights), layer4_biases)
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.479483
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 50: 0.944839
Minibatch accuracy: 62.5%
Validation accuracy: 53.3%
Minibatch loss at step 100: 0.379241
Minibatch accuracy: 93.8%
Validation accuracy: 76.3%
Minibatch loss at step 150: 0.807575
Minibatch accuracy: 75.0%
Validation accuracy: 78.4%
Minibatch loss at step 200: 0.522463
Minibatch accuracy: 87.5%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.340301
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 300: 0.784486
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
Minibatch loss at step 350: 0.477554
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 400: 0.461424
Minibatch accuracy: 93.8%
Validation accuracy: 82.7%
Minibatch loss at step 450: 0.669703
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.860903
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
M

In [5]:
batch_size = 16
patch_size = 5
depth_1 = 20
depth_2 = 40
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth_2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    max_pol = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    hidden = tf.nn.relu(tf.add(max_pol, layer1_biases))
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    max_pol = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    hidden =  tf.nn.relu(tf.add(max_pol, layer2_biases))
    shape = hidden.get_shape().as_list()
#     print(shape[1], shape[2])
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.add(tf.matmul(reshape, layer3_weights), layer3_biases))
    return tf.add(tf.matmul(hidden, layer4_weights), layer4_biases)
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
#   optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.950183
Minibatch accuracy: 6.2%
Validation accuracy: 10.6%
Minibatch loss at step 50: 1.662532
Minibatch accuracy: 37.5%
Validation accuracy: 47.7%
Minibatch loss at step 100: 0.468532
Minibatch accuracy: 87.5%
Validation accuracy: 70.1%
Minibatch loss at step 150: 1.257328
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 200: 0.650989
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 250: 0.190000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 300: 0.657097
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 350: 0.833350
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 400: 0.544277
Minibatch accuracy: 75.0%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.605553
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Minibatch loss at step 500: 0.931483
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%
M